In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from matplotlib.patches import Patch
import matplotlib.gridspec as gridspec

# --- KONFIGURASI ---
# Ganti path ini dengan lokasi file Excel/CSV kamu yang sebenarnya
file_path = r"E:\rsa-lenovo\komparasi\hasil analisa citra.xlsx"


def load_and_preprocess_data(path):
    """
    Membaca data dan mengekstrak informasi Brand dan Body Part dari Filename.
    """
    # Cek ekstensi file untuk menentukan cara baca
    if path.endswith(".csv"):
        df = pd.read_csv(path)
    else:
        df = pd.read_excel(path)

    # Fungsi kecil untuk memecah nama file
    # Asumsi format: "Body Part_brand_parameter..."
    def extract_info(filename):
        try:
            parts = filename.split("_")
            body_part = parts[0]
            brand = parts[1]
            return body_part, brand
        except:
            return "Unknown", "Unknown"

    # Terapkan ekstraksi ke kolom baru
    df[["Body Part", "Brand"]] = df["Filename"].apply(
        lambda x: pd.Series(extract_info(x))
    )

    # Bersihkan nama brand (biar konsisten huruf kecil/besar)
    df["Brand"] = df["Brand"].str.lower().str.strip()

    return df


def validate_data(df):
    """
    Validasi data untuk memastikan kelengkapan sebelum plotting.
    Returns: (is_valid, report_dict)
    """
    print("\n" + "=" * 80)
    print("🔍 VALIDASI DATA SEBELUM PLOTTING")
    print("=" * 80 + "\n")
    
    metrics = ["SNR", "CNR", "MTF50", "FWHM"]
    body_parts = df["Body Part"].unique()
    brands = df["Brand"].unique()
    
    validation_report = {
        "complete": [],
        "incomplete": [],
        "missing_brand": [],
        "missing_metrics": {}
    }
    
    print(f"📊 Total Body Parts: {len(body_parts)}")
    print(f"🏷️  Brands Found: {', '.join([b.capitalize() for b in brands])}")
    print(f"📈 Metrics: {', '.join(metrics)}\n")
    
    print("-" * 80)
    
    for part in body_parts:
        subset = df[df["Body Part"] == part]
        brands_in_part = subset["Brand"].unique()
        
        print(f"\n📦 {part}:")
        print(f"   Brands: {', '.join([b.capitalize() for b in brands_in_part])} ({len(brands_in_part)} brand(s))")
        
        # Cek apakah ada minimal 2 brand
        if len(brands_in_part) < 2:
            validation_report["incomplete"].append(part)
            validation_report["missing_brand"].append({
                "body_part": part,
                "brands": list(brands_in_part),
                "missing": [b for b in brands if b not in brands_in_part]
            })
            print(f"   ⚠️  WARNING: Hanya ada {len(brands_in_part)} brand, butuh minimal 2 untuk perbandingan!")
            print(f"   Missing: {', '.join([b.capitalize() for b in brands if b not in brands_in_part])}")
            continue
        
        # Cek missing values di setiap metrik
        missing_info = []
        for metric in metrics:
            missing_count = subset[metric].isna().sum()
            if missing_count > 0:
                missing_info.append(f"{metric}: {missing_count} missing")
                if part not in validation_report["missing_metrics"]:
                    validation_report["missing_metrics"][part] = []
                validation_report["missing_metrics"][part].append({
                    "metric": metric,
                    "count": missing_count
                })
        
        if missing_info:
            print(f"   ⚠️  Missing values: {', '.join(missing_info)}")
        else:
            print(f"   ✅ Data lengkap!")
            validation_report["complete"].append(part)
        
        # Tampilkan jumlah sample per brand
        for brand in brands_in_part:
            count = len(subset[subset["Brand"] == brand])
            print(f"      • {brand.capitalize()}: {count} samples")
    
    print("\n" + "=" * 80)
    print("📋 RINGKASAN VALIDASI:")
    print("=" * 80)
    print(f"✅ Body parts dengan data lengkap: {len(validation_report['complete'])}")
    if validation_report['complete']:
        print(f"   {', '.join(validation_report['complete'])}")
    
    print(f"\n⚠️  Body parts dengan data tidak lengkap: {len(validation_report['incomplete'])}")
    if validation_report['incomplete']:
        print(f"   {', '.join(validation_report['incomplete'])}")
    
    print("\n" + "=" * 80 + "\n")
    
    # Tanya user apakah ingin melanjutkan
    if validation_report['incomplete'] or validation_report['missing_metrics']:
        print("⚠️  PERHATIAN: Ada data yang tidak lengkap!")
        print("Plotting akan melewati body parts yang tidak lengkap.")
        print("\nSilakan cek data Anda jika ingin melengkapi sebelum plotting.")
        print("\n" + "=" * 80 + "\n")
    
    return validation_report


def generate_comparison_plots(df, output_folder, validation_report):
    """
    Membuat plot perbandingan bar chart untuk semua metrik.
    """
    # Daftar metrik yang ingin dibandingkan
    metrics = ["SNR", "CNR", "MTF50", "FWHM"]
    
    # Metrics yang lebih tinggi = lebih baik
    higher_is_better = {
        "SNR": True,
        "CNR": True, 
        "MTF50": True,
        "FWHM": False  # FWHM lebih kecil biasanya lebih baik (lebih tajam)
    }

    # Set style visualisasi agar 'bagus'
    sns.set_theme(style="whitegrid")
    
    # Gunakan warna yang lebih seimbang dan profesional
    palette = {
        "canon": "#2E86AB",      # Biru profesional
        "madeena": "#A23B72",    # Ungu/magenta - terlihat premium
    }

    print(f"📂 Hasil plot akan disimpan di: {output_folder}\n")
    print("🎨 Memulai proses plotting...\n")

    # Hanya plot body parts yang lengkap
    body_parts_to_plot = validation_report['complete']
    
    if not body_parts_to_plot:
        print("❌ Tidak ada body part dengan data lengkap untuk di-plot!")
        return

    for idx, part in enumerate(body_parts_to_plot, 1):
        print(f"[{idx}/{len(body_parts_to_plot)}] Memproses: {part}...", end=" ")
        
        subset = df[df["Body Part"] == part]
        brands_available = subset["Brand"].unique()

        # === BAR CHART LAYOUT ===
        # Setup figure dengan GridSpec untuk layout yang lebih fleksibel
        fig = plt.figure(figsize=(18, 10))
        fig = plt.figure(figsize=(16, 10))
        gs = gridspec.GridSpec(2, 3, figure=fig, hspace=0.35, wspace=0.35)
        fig.suptitle(
            f"Analisis Perbandingan Citra: {part}", 
            fontsize=24, 
            fontweight="bold", 
            y=0.98
        )

        # === BAR CHARTS (5 metrik dalam 2 baris) ===
        # === BAR CHARTS (4 metrik dalam layout 2x2) ===
        bar_positions = [(0, 0), (0, 1), (1, 0), (1, 1)]
        for i, metric in enumerate(metrics):
            row, col = bar_positions[i]
            ax = fig.add_subplot(gs[row, col])
            
            # Hitung statistik untuk setiap brand
            stats_data = []
            for brand in brands_available:
                brand_data = subset[subset["Brand"] == brand][metric]
                stats_data.append({
                    "Brand": brand.capitalize(),
                    "Mean": brand_data.mean(),
                    "Std": brand_data.std() if len(brand_data) > 1 else 0,
                    "Count": len(brand_data)
                })
            
            stats_df = pd.DataFrame(stats_data)
            
            # Buat Bar Plot dengan error bars
            # Special handling untuk FWHM (lower is better)
            if metric == "FWHM":
                # Untuk FWHM, gunakan gradient warna: hijau untuk nilai rendah (baik), merah untuk tinggi (buruk)
                min_val = stats_df["Mean"].min()
                max_val = stats_df["Mean"].max()
                colors = []
                for mean in stats_df["Mean"]:
                    if mean == min_val:  # Nilai terendah = terbaik untuk FWHM
                        colors.append("#28a745")  # Hijau (baik)
                    elif mean == max_val:  # Nilai tertinggi = terburuk untuk FWHM
                        colors.append("#dc3545")  # Merah (buruk)
                    else:
                        colors.append("#ffc107")  # Kuning (sedang)
            else:
                colors = [palette.get(b.lower(), "#888888") for b in stats_df["Brand"]]
            
            # Buat Bar Plot dengan error bars
            bars = ax.bar(
                stats_df["Brand"],
                stats_df["Mean"],
                yerr=stats_df["Std"],
                color=colors,
                capsize=8,
                alpha=0.85,
                edgecolor="black",
                linewidth=1.5
            )
            
            # Percantik Plot
            title_suffix = " [!] (Lower is Better)" if metric == "FWHM" else ""
            ax.set_title(metric + title_suffix, fontsize=14, fontweight="bold", pad=8)
            ax.set_xlabel("Brand", fontsize=10)
            ax.set_ylabel("Value", fontsize=10)
            ax.grid(axis='y', alpha=0.3, linestyle='--')
            
            # Tambahkan nilai di atas batang
            for bar, mean, std in zip(bars, stats_df["Mean"], stats_df["Std"]):
                height = bar.get_height()
                if std > 0:
                    label_text = f'{mean:.2f}\n±{std:.2f}'
                else:
                    label_text = f'{mean:.2f}'
                
                ax.text(
                    bar.get_x() + bar.get_width()/2., 
                    height + std,
                    label_text,
                    ha='center', 
                    va='bottom',
                    fontsize=9,
                    fontweight='bold'
                )

        # === LEGEND DAN NOTE ===
        ax_legend = fig.add_subplot(gs[:, 2])  # Span both rows for legend
        ax_legend.axis('off')
        
        # Custom legend
        legend_elements = [
            Patch(facecolor=palette.get(b.lower(), "#888888"), 
                  edgecolor='black', 
                  label=b.capitalize()) 
            for b in brands_available
        ]
        legend = ax_legend.legend(
            handles=legend_elements, 
            loc='upper center',
            fontsize=12,
            title="Alat",
            title_fontsize=14,
            frameon=True,
            fancybox=True,
            shadow=True
        )
        
        # Note with detailed explanations (no emojis)
        note_text = (
            "PENJELASAN METRIK:\n\n"
            "[*] SNR (Signal-to-Noise Ratio):\n"
            "    Mengukur perbandingan sinyal terhadap noise.\n"
            "    Nilai tinggi = noise lebih rendah, kualitas citra lebih baik.\n\n"
            "[*] CNR (Contrast-to-Noise Ratio):\n"
            "    Mengukur kontras citra relatif terhadap noise.\n"
            "    Nilai tinggi = kontras lebih baik dengan noise rendah.\n\n"
            "[*] MTF50 (Modulation Transfer Function):\n"
            "    Mengukur ketajaman/resolusi citra.\n"
            "    Nilai tinggi = citra lebih tajam dan detail lebih jelas.\n\n"
            "[!] FWHM (Full Width at Half Maximum):\n"
            "    Mengukur ketajaman dari lebar puncak intensitas.\n"
            "    >>> PERHATIAN: Nilai RENDAH = LEBIH BAIK! <<<\n"
            "    Grafik FWHM menggunakan warna:\n"
            "      [GREEN] = Terbaik (nilai terendah)\n"
            "      [RED] = Terburuk (nilai tertinggi)\n\n"
            "CATATAN UMUM:\n"
            "- Error bars = standar deviasi\n"
            "- SNR, CNR, MTF50: nilai tinggi lebih baik\n"
            "- FWHM: nilai RENDAH lebih baik (ditandai warna)"
        )
        ax_legend.text(
            0.3, 0.5,
            note_text,
            transform=ax_legend.transAxes,
            ha='center',
            va='top',
            fontsize=8,
            bbox=dict(boxstyle='round,pad=0.8', facecolor='lightyellow', alpha=0.4),
            family='monospace'
        )

        # Simpan gambar
        safe_filename = "".join(
            [c for c in part if c.isalpha() or c.isdigit() or c == " "]
        ).rstrip()
        file_name = f"{safe_filename}_comparison.png"
        save_path = os.path.join(output_folder, file_name)
        
        fig.savefig(save_path, bbox_inches="tight", dpi=300)
        plt.close(fig)  # Tutup figure agar tidak ditampilkan di output
        
        print(f"✅ Tersimpan: {file_name}")

    print("\n" + "=" * 80)
    print(f"🎉 Selesai! Total {len(body_parts_to_plot)} gambar telah disimpan.")
    print("=" * 80 + "\n")


# --- EKSEKUSI PROGRAM ---
try:
    print("\n" + "=" * 80)
    print("🚀 MEMULAI PROGRAM ANALISIS PERBANDINGAN CITRA")
    print("=" * 80)
    
    print("\n📂 Membaca data dari file...")
    df_result = load_and_preprocess_data(file_path)
    print(f"✅ Data berhasil dibaca: {len(df_result)} baris")

    # Tampilkan preview data
    print("\n📋 Preview Data:")
    print("-" * 80)
    display(df_result[["Filename", "Body Part", "Brand", "Score"]].head(10))
    
    # VALIDASI DATA TERLEBIH DAHULU
    validation_report = validate_data(df_result)
    
    # Jika ada data yang lengkap, lanjutkan ke plotting
    if validation_report['complete']:
        # Menentukan folder output
        base_dir = os.path.dirname(os.path.abspath(file_path))
        output_dir = os.path.join(base_dir, "hasil_plot_perbandingan")
        os.makedirs(output_dir, exist_ok=True)
        
        # Jalankan visualisasi
        generate_comparison_plots(df_result, output_dir, validation_report)
        
        print(f"\n📁 Semua gambar telah disimpan di:\n   {output_dir}")
    else:
        print("\n❌ Tidak dapat melanjutkan plotting karena tidak ada data yang lengkap.")
        print("Silakan perbaiki data Anda terlebih dahulu.\n")

except FileNotFoundError:
    print(f"\n❌ Error: File tidak ditemukan di path: {file_path}")
    print("Pastikan nama file dan path sudah benar.\n")
except Exception as e:
    print(f"\n❌ Terjadi kesalahan: {e}")
    import traceback
    traceback.print_exc()



🚀 MEMULAI PROGRAM ANALISIS PERBANDINGAN CITRA

📂 Membaca data dari file...
✅ Data berhasil dibaca: 48 baris

📋 Preview Data:
--------------------------------------------------------------------------------


,Filename,Body Part,Brand,Score
0,Ankle AP_canon_52kV_10mA_5s.jpg,Ankle AP,canon,0.363324
1,Ankle AP_madeena_80kV_50mA_0.4s.jpg,Ankle AP,madeena,0.474990
2,Ankle Lateral_canon_52kV_10mA_5s.jpg,Ankle Lateral,canon,0.432315
3,Ankle Lateral_madeena_80kV_50mA_0.4s.jpg,Ankle Lateral,madeena,0.418329
4,Antebrachi AP_canon_48kV_11.11mA_4.5s.jpg,Antebrachi AP,canon,0.210001
5,Antebrachi AP_madeena_70kV_50mA_0.2s.jpg,Antebrachi AP,madeena,0.592742
6,Antebrachi Lateral_canon_48kV_11.11mA_4.5s.jpg,Antebrachi Lateral,canon,0.385774
7,Antebrachi Lateral_madeena_70kV_50mA_0.2s.jpg,Antebrachi Lateral,madeena,0.462358
8,Cruris AP_canon_55kV_7.14mA_7s.jpg,Cruris AP,canon,0.399787
9,Cruris AP_madeena_80kV_50mA_0.5s.jpg,Cruris AP,madeena,0.495457



🔍 VALIDASI DATA SEBELUM PLOTTING

📊 Total Body Parts: 24
🏷️  Brands Found: Canon, Madeena
📈 Metrics: SNR, CNR, MTF50, FWHM

--------------------------------------------------------------------------------

📦 Ankle AP:
   Brands: Canon, Madeena (2 brand(s))
   ✅ Data lengkap!
      • Canon: 1 samples
      • Madeena: 1 samples

📦 Ankle Lateral:
   Brands: Canon, Madeena (2 brand(s))
   ✅ Data lengkap!
      • Canon: 1 samples
      • Madeena: 1 samples

📦 Antebrachi AP:
   Brands: Canon, Madeena (2 brand(s))
   ✅ Data lengkap!
      • Canon: 1 samples
      • Madeena: 1 samples

📦 Antebrachi Lateral:
   Brands: Canon, Madeena (2 brand(s))
   ✅ Data lengkap!
      • Canon: 1 samples
      • Madeena: 1 samples

📦 Cruris AP:
   Brands: Canon, Madeena (2 brand(s))
   ✅ Data lengkap!
      • Canon: 1 samples
      • Madeena: 1 samples

📦 Cruris Lateral:
   Brands: Canon, Madeena (2 brand(s))
   ✅ Data lengkap!
      • Canon: 1 samples
      • Madeena: 1 samples

📦 Elbow Joint AP:
   Brands:

C:\Users\adlan\AppData\Local\Temp\ipykernel_6432\2290427956.py:181: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(16, 10))


✅ Tersimpan: Thorax Lateral Kiri_comparison.png
[21/24] Memproses: Thorax Lordotix... 

...

<Figure size 1800x1000 with 0 Axes>

<Figure size 1800x1000 with 0 Axes>

<Figure size 1800x1000 with 0 Axes>

<Figure size 1800x1000 with 0 Axes>

<Figure size 1800x1000 with 0 Axes>